# 2. Pandas

In this exercise, we will be looking at pandas, a Python library that provides many useful tools for loading, displaying, and cleaning data.  

To aid us in showing off the functionality of this library, we will be looking at the MetObjects dataset, which comes courtesy of the Metropolitan Museum of Art in New York [https://www.metmuseum.org/]. 


## 2.1 Downloading the dataset
You can use a leading `!` in a line of Jupyter notebook code to specify that the rest of the line should be interpreted as a shell command. This is convenient for modifying files or running scripts that live on your filesystem without having to switch between the browser and terminal. Let's use this syntax to download the Met Museum dataset

In [ ]:
dataset_url = ''   # Normal python code


# Jupyter notebook "magic" lines prepended with a ! character

# Dataset 
MET_DATA_PATH = "./data/metmuseum/MetObjects.csv" 

In [ ]:
# Module imports and plot settings
import pandas as pd
import matplotlib.pyplot as plt


pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None 
plt.rcParams["figure.figsize"] = [8, 7]
plt.rcParams["figure.autolayout"] = True






## 2.2 The DataFrame

The DataFrame is the central data structure provided by Pandas, and it is this structure that we need to interrogate when we want to ask questions about our data. You can think of a DataFrame as a table with rows of records and columns that describe the fields of those records. Pandas provides built in functions for loading text files and automatically puts their contents into a DataFrame. The dataset we just downloaded (`MetObjects.csv`) is a CSV (comma separated value) file, so we need to use the `load_csv` function provided by Pandas.

In [ ]:
# Taking a look at the dataset


## 2.3 Accessing and displaying data

### 2.3.1 Integer indexing
Similar to Python list slices, uses 0-indexed start and end positions to return a subset of the dataframe. With a Padas dataframe, this is done via the `iloc` indexer.

In [ ]:
# Get rows number 29 to 35


### 2.3.2 Boolean Series
A series is a 1-D array - a boolean series is one that is filled with boolean (i.e., `True` or `False`) values. We can pass boolean series into a Dataframe's `loc` indexer to keep only the values that align with `True`. Different boolean series of the same length can be combined using the following logical operators: `&` (and), `|` (or), `~` (not).

In [ ]:
# getting all rows from the 'Medieval Art' department


In [ ]:
# getting rows from the 'Medieval Art' OR 'European Sculpture and Decorative Arts' departments


### 2.3.3 Grouping by column name(s)
We can also group the data by a list of columns. This returns a Pandas GroupBy object, which contains a dictionary of mappings from each group name to a Series of its elements

### 2.3.4 Using `where`
Similar to Numpy arrays, Pandas dataframes also make use of the `where` function to conditionally modify its elements based on some criteria. `where` takes a dataframe condition as an argument and returns the modified dataframe - if the condition is fulfilled, it keeps the value of the field, if not, it replaces it with `NaN`.

## 2.4 Data cleaning

We can see that there are several problems with the dataset right off the bat:
1. First row seems to contain garbage: none of the column names match up with the data types, and many are NaN
2. It looks like many of the columns are completely empty - they add nothing to the dataset but clutter it
3. Too many columns! This depends on what your needs are, but we don't need all of them for this exercise
4. Inconsistent formatting in the Dimensions column - makes it difficult to use them downstream
5. Mixed datatypes in Year fields

Let's address all of these issues one by one

### 2.4.1 Deleting rows by index
We can get rid of the first row (index 0) by taking a slice of the dataframe beginning at index 1 and going all the way to the end.

In [ ]:
# Get rid of the first row, we can use dataframe slicing to accomplish this:



### 2.4.2 Removing columns

Columns can be removed conditionally by checking their contents to see if they meet a certain criteria, or simply by name

In [ ]:
# Get rid of all columns that are completely empty


In [ ]:
# At this point, we can also drop the columns that are irrelevant to our needs
exclude_cols = [
    "Country",
    "Culture",
    "Is Highlight",
    "Is Timeline Work",
    "Object End Date",
    "Gallery Number",
    "Period",
    "Constituent ID",
    "Artist Role",
    "Artist Prefix",
    "Artist Display Name",
    "Artist Display Bio",
    "Artist Suffix",
    "Artist Alpha Sort",
    "Artist Gender",
    "Artist Nationality",
    "Artist ULAN URL",
    "Artist Wikidata URL",
    "Credit Line",
    "Object ID",
    "Geography Type",
    "City",
    "State",
    "County",
    "Region",
    "Classification",
    "Rights and Reproduction",
    "Link Resource",
    "Object Wikidata URL",
    "Repository",
    "Tags AAT URL",
    "Tags Wikidata URL",
    "Artist Begin Date",
    "Artist End Date",
    "Object Date",
]



### 2.4.3 Removing rows
We can also get rid of rows that do not meet certain criteria. For example, given a subset of fields that we deem very important, we can drop all rows are NaN in any of these fields

In [ ]:
# Dropping records (rows)


### 2.4.4 Applying functions to columns

We noted earlier that the Dimensions field is a bit messy (inconsistent mixing of imperial and metric units).  We need it to have consistent formatting so that any functions we write later can work with the values without any complicated processing. Doing the complicated work up front saves a lot of time down the line! Don't worry too much about how exactly this function works. This is just to show that we can write arbitrarily complex cleaning functions and apply it to a Dataframe's columns. The important thing to note about this function is that if it can't find a suitable dimension to extract for whatever reason, **it will fail, and on failure will return NaN** (not a number).  

When this function is applied to the Dimensions column, the column will be left with values that look like:
1. a OR
2. a, b OR
3. a, b, c OR
4. NaN

where a, b and c are lengths in centimetres and NaN is an indication that the extraction function has failed

We can apply this function using the DataFrame's `apply` method. This takes the function to be applied, along with any additional arguments (In addition to the value of the column, of course)

In [ ]:
import re

import numpy as np

# Defining the function that will be used to extract metric dimensions. The first argument to
# the function must be the value in the column. 
def extract_dimensions(dim_str):
    dimensions_exp = r"(?<=\()( *\d.+?)(?=cm\))"
    delimiter_exp = r"(?:\d+\.?\d* *)([^\.\n])(?: *\d+\.?\d* *)?(?:[^\.\n] *\d+\.?\d*)?"
    retval = np.nan
    try:
        dim_str = dim_str.split("\n")[0]
        dimensions = re.search(dimensions_exp, dim_str).group(0)
        delimiter = re.search(delimiter_exp, dimensions.strip()).groups()[0]
        if not delimiter.isnumeric():
            # There are multiple dimensions
            retval = ','.join(dimensions.split(delimiter))
        else:
            retval = dimensions
    except AttributeError as e:
        pass
    finally:
        return retval

In [ ]:
# Standardise the dimensions of the pieces

# Because extract_dimensions can fail and return NaN, we also need to remove any records where Dimensions is NaN


### 2.4.5 Default values
Artists are known to often leave their work untitled (why??). In our dataset, this is not handled very gracefully - the titles of such artworks are simple NaN. Fortunately, we have another way of dealing with missing data: assinging a default value. We can replace any instance of a NaN title with the string "Untitled"

In [ ]:
# Setting default values. 



### 2.4.6 Data types
We change some data types that don't really make sense: `AccessionYear` and `Object Begin Date` were originally loaded in with mixed datatypes (some are strings, some are numbers), which makes it difficult to sort correctly.

In [ ]:

# Finally, we can sort a dataframe based on the values in a specific column


## 2.5 Dataframe interrogation
We can now begin to ask some interesting questions about this dataset:
1. Which department houses the oldest artwork in the museum? Use `Object Begin Date` for this task.
2. What is the proportion of artworks from each department? Display this graphically using `pd.DataFrame.plot.pie`
3. What is the average area in $cm^2$ of Paintings in the Asian Art department?
4. _What is the most common theme across all the paintings? Or, which tag is most common?_
5. _How many artworks were delivered to the museum each decade, and what department took them?_


### 2.5.1 Exercise
Oldest artwork in the museum 



In [ ]:
# Sort the dataset by `Object Begin Date`  

# Get the first element 

### 2.5.2 Exercise
Proportion of artworks from each department

In [ ]:
# Group the dataset by `Department`

# Get the sizes of each grouping and put them in a coulumn called `Counts` 


In [ ]:
# draw as pie chart - Just run this cell to see the output of your grouping :)
departments.plot.pie(y="Counts", 
                     explode=[0.125 for _ in range(len(departments))], 
                     labels=departments['Department'], 
                     legend=None, ylabel="")

### 2.5.3 Question 3
Let's tackle Q3: What is the average area in $cm^2$ of Paintings in the Asian wing? 

In [ ]:
# Select the pieces that belong to the Asian Art department AND are paintings
asian_art_bool_series = ...
painting_bool_series = ...
asian_paintings = ...
asian_paintings

In [ ]:
import math

# Define an area calculation function to each item in the Dimensions column
def calc_area(dim_string):
    # Note that some paintings are listed as having only 1 dimension - here we can assume that they are 
    # circular and that the given dimension is its diameter (and probably not radius as that cannot be measured as easily)
    
    # extract dimensions from the string and place them into a list
    dims_separated = ... 
    if len(dims_separated) == 1:
        # We have a circular painting, apply the circular area formula
        diameter = ...  # dimensions are strings, so need to be converted to floats
        radius = ...
        area = ...
        return area
    elif len(dims_separated) >= 2:
        # We have a rectangular painting, use the rectangular area formula
        # Some paintings probably have a 3rd dimension to indicate the depth
        # of its frame, we can ignore this and only take the first 2 elements
        width  = ...
        height = ...
        area = ...
        return area

Test out your function here on some random dimension strings to see if it works! Once you're satisfied that you've got it working, you can move on. If you want some inspiration for inputs, use these:
* "83, 99, 3"
* "10"
* "abc"
* "55, 2"

In [ ]:
# Function testing

In [ ]:
# Apply your function to the DataFrame and assign its results to a new column `Area` within
# the same dataframe
asian_paintings['Area']  = ...
asian_paintings

In [ ]:
# Get the mean of the `Area` column
average_area = ...
average_area

### 2.5.4 Question 4
What is the most common theme across all the paintings? Or, which tag is most common?

In [ ]:
# declare an empty list to collect all the separated tags in the database
tags_list = ...

In [ ]:
# get the Tags series out of the dataset and assign it to a variable
# called tags_series
tags_series = ...

In [ ]:
# Define function:
def update_list(t, l):
    pass

# Apply function to tags_series



In [ ]:
# Use the tags list to instantiate a dataframe called tags_df
tags_df = ...



In [ ]:
# group the dataframe by Tag and count the occurences of each tag
tags_df = ...



In [ ]:
# sort the list by count to find the one with the tag with the 
# highest number of occurences
tags_df = ...